<div align="center">
<img height="80" src="https://miro.medium.com/max/198/1*-ZlGSSXvRbDSr13_CDRaJA.png">
<p style="text-align:center">Accelerate the development of ML applications with modern workflow.<br>
Go from experiments to real-world applications faster at scale.<br>
</div>

<div align="center">
    <a target="_blank" href="https://join.slack.com/t/vessl-ai-community/shared_invite/zt-1a6schu04-NyjRKE0UMli58Z_lthBICA"><img src="https://img.shields.io/badge/style--5eba00.svg?label=Slack&logo=slack&style=social"></a>&nbsp;
    <a target="_blank" href="https://twitter.com/vesslai"><img src="https://img.shields.io/badge/style--5eba00.svg?label=Twitter&logo=twitter&style=social"></a>&nbsp;
    <a target="_blank" href="https://www.linkedin.com/company/vesslai"><img src="https://img.shields.io/badge/style--5eba00.svg?label=LinkedIn&logo=linkedin&style=social"></a>&nbsp;
    <a target="_blank" href="https://vesslai.medium.com/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=Medium&logo=medium&style=social"></a>&nbsp;
    <a target="_blank" href="https://github.com/vessl-ai/examples/"><img src="https://img.shields.io/badge/style--5eba00.svg?label=GitHub&logo=github&style=social"></a>&nbsp;
    <br>
</div>

## Quickstart
 
**Use VESSL Workspace to build models from scratch** <br>
⚡ &nbsp; Get instant access to GPU-accelerated Jupyter notebook <br>
🎛️ &nbsp; Jump right into pre-configured dev environment shared across your team <br>
📈 &nbsp; log and visualize experiments a dashboard with `vessl` library<br>

## 1. Install `vessl` library and login

All notebooks created on VESSL Workspace come with the latest version of `vessl` library. 

In [ ]:
# Check current version of vessl
!vessl --version

Let's start by logging into your account and configuring your organization and project. This designates where your experiments will be recorded.

In [ ]:
# Create a new project to run this example
import datetime
import vessl

project_name = f"vessl-example-{int(datetime.datetime.now().timestamp())}"
vessl.create_project(project_name)

In [ ]:
# Configure default project
vessl.configure(project_name=project_name)

These notebooks also come with essential ML libraries like `matplotlib` and `torch` pre-installed, which you can view by `pip list` or using the `watermark` library. You can configure these default libraries by [🔗&nbsp; building custom Docker images](https://docs.vessl.ai/user-guide/workspace/building-custom-images#building-from-community-maintained-images) for your team. 

In [ ]:
#!!pip list

# Show the current versions of the pre-installed libraries
!!pip install watermark
import watermark
%load_ext watermark
%watermark -v -p numpy,matplotlib,torch,torchvision

Before moving on to our dataset and model, let's import these libraries.

In [ ]:
# Import required libraries
import os
import matplotlib
import matplotlib.pyplot as plt
import torch
from torch import optim
import torch.nn.functional as F
from torch.optim.lr_scheduler import StepLR

%pylab
%matplotlib inline

## 2. Prepare the dataset

In this example, we will train an image classification model using the [🔗&nbsp; MNIST dataset](https://paperswithcode.com/dataset/mnist). Here, we will use the publicly available dataset from `torchvision`. You can also mount a volume and import your own datasets using [🔗&nbsp; VESSL Dataset](https://docs.vessl.ai/user-guide/dataset).

In [ ]:
# Download the MNIST training data
from torchvision import datasets
from torchvision.transforms import ToTensor
train_data = datasets.MNIST(
    root = '/tmp/example-data',
    train = True,                         
    transform = ToTensor(), 
    download = True,            
)
test_data = datasets.MNIST(
    root = '/tmp/example-data', 
    train = False, 
    transform = ToTensor()
)

# Downsize the dataset to run fast.
train_data.data = train_data.data[:200]
test_data.data = test_data.data[:1000]
print(f'The shape of train data: {train_data.data.shape}')
print(f'The shape of test data: {test_data.data.shape}')

In [ ]:
# Render sample images from the training data
figure = plt.figure(figsize=(10, 8))
cols, rows = 5, 1

for i in range(1, cols*rows + 1):
    sample_idx = torch.randint(len(train_data), size=(1,)).item()
    img, label = train_data[sample_idx]
    figure.add_subplot(rows, cols, i)
    plt.title(label)
    plt.axis("off")
    plt.imshow(img.squeeze(), cmap="gray")

plt.show()

In [ ]:
# Define data loader
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(train_data, batch_size=128, shuffle=True, num_workers=1)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=128, shuffle=False, num_workers=1)

## 3. Define the model

Let's define an image classification model using a simple two-dimensional convolutional neural network.

In [ ]:
# Define model
import torch.nn as nn

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, 3, 1)
        self.conv2 = nn.Conv2d(32, 64, 3, 1)
        self.relu = nn.ReLU()
        self.pool = nn.MaxPool2d(2)
        self.drop1 = nn.Dropout2d(0.25)
        self.drop2 = nn.Dropout2d(0.5)
        self.fc1 = nn.Linear(9216, 128)
        self.fc2 = nn.Linear(128, 10)
        self.softmax = nn.LogSoftmax(1)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.relu(self.conv2(x))
        x = self.pool(x)
        x = self.drop1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = self.relu(x)
        x = self.drop2(x)
        x = self.fc2(x)
        return self.softmax(x)

In [ ]:
# Initialize model
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = CNN().to(device)
if torch.cuda.device_count() > 1:
    model = nn.DataParallel(model)

print(model)

## 4. Use `vessl.log()` to log and visualize experiments

Now that we have the dataset and model ready, the next step is to write a script for training and testing the model. With VESSL, you can record outputs from your experiments such as key metrics and media files. Store, visualize, and compare experiment results using [🔗&nbsp; `vessl.log()`](https://docs.vessl.ai/api-reference/python-sdk/utils/vessl.log).

In [ ]:
# Train function with VESSL logging
from torch.autograd import Variable

def train(model, device, train_loader, optimizer, epoch, start_epoch):
    model.train()
    loss = 0
    for batch_idx, (data, label) in enumerate(train_loader):
        data, label = data.to(device), label.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, label)
        loss.backward()
        optimizer.step()
        if batch_idx % 128 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch + 1, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))

    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        epoch + 1, len(train_loader.dataset), len(train_loader.dataset), 100, loss.item()))

    # Logging loss metrics to Vessl
    vessl.log(
        step=epoch + start_epoch + 1,
        payload={'loss': loss.item()}
    )

In [ ]:
# Test function (with vessl plot uploading)
def test(model, device, test_loader, save_image):
    model.eval()
    test_loss = 0
    correct = 0
    test_images = []
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            test_images.append(vessl.Image(
                data[0], caption="Pred: {} Truth: {}".format(pred[0].item(), target[0])))

    test_loss /= len(test_loader.dataset)
    test_accuracy = 100. * correct / len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset), test_accuracy))

    # Saving inference results with caption into Vessl
    if save_image:
        vessl.log({"Examples": test_images})

    return test_accuracy

## 5. Train the model

Let's run the experiment. [🔗&nbsp; `vessl.init()`](https://docs.vessl.ai/api-reference/python-sdk/utils/vessl.init) creates a new experiment and our Python SDK will automatically record all metrics and logs on the the newly created experiment. You can check the progress on VESSL by visiting the output link.

In [ ]:
# Initialize new experiment via VESSL SDK 
vessl.init()

In [ ]:
# Hyperparameters
epochs = 5
batch_size = 128
learning_rate = 0.01
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

# Train the model
for epoch in range(epochs):
    train(model, device, train_loader, optimizer, epoch, 0)
    test(model, device, test_loader, True)

    scheduler.step()

In [ ]:
# Finish experiment
vessl.finish()

## What's next

**Try `vessl run`** <br>
Just add `vessl run` before your command to run the process on VESSL infrastructure. <br>
Try `vessl run python vessl-example-mnist.py` on terminal.


**Use VESSL Experiments to orchestrate experiments and optimize your model** <br>
🪄 &nbsp; Orchestrate multiple experiments in parallel using [🔗&nbsp; VESSL experiments](https://docs.vessl.ai/user-guide/experiment) <br>
🏃 &nbsp; Move seemlessly between local, cloud, and on-premise servers using [🔗&nbsp; `vessl run`](https://docs.vessl.ai/api-reference/cli/vessl-run#overview) <br>
🔥 &nbsp; Optimize your model using [🔗&nbsp; hyperparameter optimization](https://docs.vessl.ai/user-guide/sweep) and [🔗&nbsp; distributed training](https://docs.vessl.ai/user-guide/experiment/distributed-experiments)